## leNet 5 模型

导入回调函数以保存网络生成文件

In [1]:

import mindspore.nn as nn
from mindspore.train import Model

from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet
from mindvision.engine.callback import LossMonitor

epochs = 10  # 训练轮次

# 1. 构建数据集
# download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
# dataset_train = download_train.run()

# 2. 定义神经网络
network = lenet(num_classes=10, pretrained=False)
# 3.1 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 3.2 定义优化器函数
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)
# 3.3 初始化模型参数
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'accuracy'})

# 4. 对神经网络执行训练
# model.train(epochs, dataset_train, callbacks=[LossMonitor(0.01, 1875)])


In [2]:
import numpy as np
import mindspore
import mindspore.nn as nn
from mindspore import Tensor,Model
from mindspore import  dtype as mstype
from mindspore.train.callback import LossMonitor
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset as ds
import os
import argparse


parser = argparse.ArgumentParser(description='MindSpore LeNet Example')
parser.add_argument('--device_target', type=str, default="CPU", choices=['Ascend', 'GPU', 'CPU'])


DATA_DIR = "./datasets/MNIST_Data/train"
# DATA_DIR = "./datasets/cifar-10-batches-bin/train"

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
#采样器
sampler = ds.SequentialSampler(num_samples=5)

# dataset = ds.Cifar100Dataset(DATA_DIR,sampler=sampler)

dataset = ds.MnistDataset(DATA_DIR,sampler=sampler) #这个是MNIST数据集
# dataset = ds.Cifar10Dataset(DATA_DIR,sampler=sampler) #这个是Cifar10数据集

class LeNet5(nn.Cell):
    def __init__(self,num_class=10,num_channel = 1):
    # 初始化网络
        super(LeNet5,self).__init__()
        # 定义所需要的运算
        self.conv1 = nn.Conv2d(num_channel,6,5,pad_mode='valid') # 卷积
        self.conv2 = nn.Conv2d(6,16,5,pad_mode='valid')
        self.fc1 = nn.Dense(256,120) # 全连接层
        # self.fc1 = nn.Dense(16*5*5,120) # 全连接层
        self.fc2 = nn.Dense(120,84)
        self.fc3 = nn.Dense(84,num_class)
        self.max_pool2d = nn.MaxPool2d(kernel_size=2,stride=2)# 最大池化-降采样
        self.relu = nn.ReLU() # 激活函数
        self.flatten = nn.Flatten()# flatten 扁平的意思=> 将原来的高维数组换成只有 一行 的数组 列数是之前的各维度之积

    # 定义网络构建函数
    def construct(self,x):
        # 构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x    

#初始化网路
net = LeNet5()
for m in net.parameters_and_names():
    print(m)
    
#定义超参
epoch = 12
batch_size = 64
learning_rate = 0.1
 
#构建数据集
sampler = ds.SequentialSampler(num_samples=128)
dataset = ds.MnistDataset(DATA_DIR,sampler = sampler)

#数据类型的转换
type_cast_op_image = C.TypeCast(mstype.float32)
type_cast_op_label = C.TypeCast(mstype.int32)

#数据序列读取方式
HWC2CHW = CV.HWC2CHW()

#构建数据集
dataset = dataset.map(operations=[type_cast_op_image,HWC2CHW],input_columns="image")
dataset = dataset.map(operations=type_cast_op_label,input_columns="label")
dataset = dataset.batch(batch_size)

print("\n")
#传入定义的超参
for p in net.trainable_params():
    print(p)

#函数
optim = nn.SGD(params=net.trainable_params(),learning_rate=learning_rate)# 自动微分反向传播
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean') # 交叉熵损失函数

('conv1.weight', Parameter (name=conv1.weight, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True))
('conv2.weight', Parameter (name=conv2.weight, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True))
('fc1.weight', Parameter (name=fc1.weight, shape=(120, 256), dtype=Float32, requires_grad=True))
('fc1.bias', Parameter (name=fc1.bias, shape=(120,), dtype=Float32, requires_grad=True))
('fc2.weight', Parameter (name=fc2.weight, shape=(84, 120), dtype=Float32, requires_grad=True))
('fc2.bias', Parameter (name=fc2.bias, shape=(84,), dtype=Float32, requires_grad=True))
('fc3.weight', Parameter (name=fc3.weight, shape=(10, 84), dtype=Float32, requires_grad=True))
('fc3.bias', Parameter (name=fc3.bias, shape=(10,), dtype=Float32, requires_grad=True))


Parameter (name=conv1.weight, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True)
Parameter (name=conv2.weight, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True)
Parameter (name=fc1.weight, shape=(120, 256), dtype=Float32, requir

In [3]:
# 引入callback函数进行模型的保存
from mindspore.train.callback import ModelCheckpoint
ckpt_cb = ModelCheckpoint()

In [4]:
# 根据需求对checkpoint函数进行配置
from mindspore.train.callback import ModelCheckpoint,CheckpointConfig

# save_checkpoint_steps=32 每32次保存一次
# keep_checkpoint_max=10 最多保留10个checkpoint

config_cb = CheckpointConfig(save_checkpoint_steps=4,keep_checkpoint_max=10)

# prefix = 'lenet5' 定义checkpoint文件前缀名称
# directory 指定保存的地址 

ckpt_cb = ModelCheckpoint(prefix='lenet5',directory="../quickStart01/model/",config=config_cb)

In [5]:
#开始训练：输入训练轮次和训练数据集
model = Model(net,loss_fn=loss,optimizer=optim)
model.train(epoch=epoch,train_dataset=dataset,callbacks=ckpt_cb) 

[WARNING] ME(6188:16984,MainProcess):2022-04-29-20:04:19.514.641 [mindspore\train\model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


![](1651218789000.png)

其中  

\-graph.meta  是编译后的计算图

.ckpt               是checkpoint文件，是权重文件

![](1651218704000.png)

## 加载模型
#### 进行 重训练 或者 推理
要加载模型权重 需要 先创建 相同模型 的实例

然后使用`load_checkpoint`和`load_param_into_net`方法加载参数

In [6]:
# 创建相同的模型实例
from mindspore import load_checkpoint,load_param_into_net

net0 = LeNet5()

model0  = Model(net0,loss_fn = loss,metrics={"accuracy"})

# 将模型参数存入字典
param_dict = load_checkpoint("../quickStart01/model/lenet5_1-12_2.ckpt")
# print(param_dict)
# 将参数加载到网络中去
load_param_into_net(net0,param_dict)

[]

## 模型验证
针对仅推理场景，把参数直接加载到网络中，以便后续的推理验证

In [7]:
# create_custom_dataset
TEST_DIR = DATA_DIR[:-5]+"test"
# print(TEST_DIR)
# 定义验证数据集
sampler = ds.SequentialSampler(num_samples=128)
dataset_eval = ds.MnistDataset(TEST_DIR,sampler=sampler)
# dataset_eval = create_dataset(TEST_DIR,32,1)

#数据类型的转换
type_cast_op_image = C.TypeCast(mstype.float32)
type_cast_op_label = C.TypeCast(mstype.int32)

#数据序列读取方式
HWC2CHW = CV.HWC2CHW()

#构建数据集
dataset_eval = dataset_eval.map(operations=[type_cast_op_image,HWC2CHW],input_columns="image")
dataset_eval = dataset_eval.map(operations=type_cast_op_label,input_columns="label")
dataset_eval = dataset_eval.batch(batch_size)

# 调用eval()进行推理
acc = model0.eval(dataset_eval)
print(acc)

[WARNING] ME(6188:16984,MainProcess):2022-04-29-20:04:20.883.610 [mindspore\train\model.py:954] CPU cannot support dataset sink mode currently.So the evaluating process will be performed with dataset non-sink mode.


{'accuracy': 0.1171875}


## 迁移学习

针对任务中断及微调（Fine-tuning）场景 可以加载网络参数和优化器参数到模型中。

如下：

In [8]:
# 设置训练轮次

epoch00 = 1

# 定义训练数据集
dataset00 = ds.MnistDataset(DATA_DIR,sampler = sampler) 

# 调用train()进行训练
model0.train(epoch00,dataset00)

[WARNING] ME(6188:16984,MainProcess):2022-04-29-20:04:21.116.613 [mindspore\train\model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


TypeError: mindspore\core\utils\check_convert_utils.cc:664 _CheckTypeSame] For primitive[Conv2D], the input type must be same.
name:[w]:Ref[Tensor(F32)].
name:[x]:Tensor[UInt8].

The function call stack (See file 'f:\000Codefield\CODE_PY\AI\jupyterNotebook\MindSpore01\quickStart01\rank_0\om/analyze_fail.dat' for more details):
# 0 In file D:\Anaconda3\envs\mindsp161\lib\site-packages\mindspore\nn\wrap\cell_wrapper.py(110)
        out = self._backbone(data)
              ^
# 1 In file C:\Users\Administrator\AppData\Local\Temp\ipykernel_6188\45943137.py(49)
# 2 In file D:\Anaconda3\envs\mindsp161\lib\site-packages\mindspore\nn\layer\conv.py(267)
        if self.has_bias:
# 3 In file D:\Anaconda3\envs\mindsp161\lib\site-packages\mindspore\nn\layer\conv.py(266)
        output = self.conv2d(x, self.weight)
                 ^
